# Coastal flood hazard workflow

In [ ]:
# Import modules
import subprocess
from pathlib import Path

from hydroflows import Workflow, WorkflowConfig
from hydroflows.methods import coastal
from hydroflows.utils.example_data import fetch_data


In [ ]:
# Fetch the global build data
cache_dir = fetch_data(data="global-data")


In [ ]:
# Define case name and root directory
name = "coastal_events"
pwd = Path().resolve()  # Get the current file location
case_root = Path(pwd, "cases", name)  # output directory

In [ ]:
# Setup the configuration
config = WorkflowConfig(
    region=Path(pwd, "data/build/region.geojson"),
    gtsm_catalog=Path(cache_dir, "data_catalog.yml"),
    start_time="2014-01-01",
    end_time="2021-12-31",
    rps=[2, 5, 10],
)

wf = Workflow(config=config, name=name, root=case_root)


In [ ]:
# Get the GTSM data
get_gtsm_data = coastal.GetGTSMData(
    gtsm_catalog=wf.get_ref("$config.gtsm_catalog"),
    start_time=wf.get_ref("$config.start_time"),
    end_time=wf.get_ref("$config.end_time"),
    region=wf.get_ref("$config.region"),
    data_root="data/gtsm",
)
wf.create_rule(get_gtsm_data, rule_id="get_gtsm_data")


In [ ]:
# Generate coastal design events
coastal_design_events = coastal.CoastalDesignEvents(
    surge_timeseries=get_gtsm_data.output.surge_nc,
    tide_timeseries=get_gtsm_data.output.tide_nc,
    bnd_locations=get_gtsm_data.output.bnd_locations,
    rps=wf.get_ref("$config.rps"),
    event_root="data/events",
)
wf.create_rule(coastal_design_events, rule_id="coastal_design_events")


In [ ]:
# Test the workflow
wf.dryrun()


In [ ]:
# Write the workflow to a Snakefile
wf.to_snakemake()

# show the files in the case directory
print(f"{wf.root.relative_to(pwd)}:")
for f in wf.root.iterdir():
    print(f"- {f.name}")

In [ ]:
from IPython.display import SVG

# (test) run the workflow with snakemake and visualize the directed acyclic graph
# make sure to have snakemake installed in your environment
subprocess.run('snakemake --dag | dot -Tsvg > dag.svg', cwd=wf.root, shell=True).check_returncode()

# show the dag
SVG(Path(wf.root, "dag.svg").as_posix())